In [1]:
import numpy as np
import h5py

In [2]:
###############
# Input - Fermionic object in tau domain
#
# Ouput - Fermionic object in Matsubara domain at n = 0
###############
def sigma_zero(Sigma):
    # Setup
    beta = 100
    nts, ncheb = np.shape(Sigma)[0], np.shape(Sigma)[0] - 2
    tau_mesh = np.zeros(nts)
    tau_mesh[0], tau_mesh[-1] = 0, beta
    for it in range(1,nts-1):
        z = np.cos(np.pi*(it - 0.5)/ncheb)
        tau_mesh[nts - it - 1] = (z+1)*beta/2
    # Define Chebyshev polynomials
    _Ttc = np.zeros((nts, ncheb))
    for it in range(nts):
        x = 2.0 * tau_mesh[it]/beta - 1.0
        _Ttc[it, 0] = 1.0
        _Ttc[it, 1] = x
        for ic in range(2,ncheb):
            _Ttc[it, ic] = 2.0*x*_Ttc[it, ic-1] - _Ttc[it, ic-2]

    x = 1.0/ncheb
    _Tct = np.zeros((ncheb, nts))
    for ic in range(ncheb):
        if ic == 0:
            factor = 1.0
        else:
            factor = 2.0
        for it in range(1,nts-1):
            _Tct[ic, it] = _Ttc[it, ic] * factor * x

    _T_0l = np.zeros(ncheb, dtype=complex)
    Tnl = h5py.File('/Users/CanonYeh/Projects/chebyshev_input/TNC.h5')
    for ic in range(ncheb):
        re = Tnl["TNC_"+str(ic)+"_r"][()]
        im = Tnl["TNC_"+str(ic)+"_i"][()]
        _T_0l[ic] = complex(re[0],im[0]) * beta/20
    Tnl.close()

    # Selfenergy in Chebyshev representation
    Sigma_c = np.einsum('ij,jklmn -> iklmn', _Tct, Sigma)
    # Selfenergy in zero frequency
    Sigma_w0 = np.einsum('i,ijklm->jklm', _T_0l, Sigma_c)

    return(Sigma_w0)